## GeoTIFFs to single netCDF with time dimension enabled

---
Convert CHIRPS GeoTIFF in a folder to single NetCDF file with time dimension enabled that is CF-Compliant
http://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html
 
Based on Rich Signell's answer on StackExchange: https://gis.stackexchange.com/a/70487
This script was tested using CHIRPS dekad data. Adjustment is needed if using other timesteps data for CHIRPS
NCO (http://nco.sourceforge.net) must be installed before using this script
 
Modified by
Benny Istanto, UN World Food Programme, benny.istanto@wfp.org

---

### CHIRPS Java Monthly

Case Java island, Indonesia - 105.05,116,25,-8.80,-5.05

Original data in GeoTIFF format downloaded from https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_monthly/tifs/
Then clipped using Java boundary (https://www.dropbox.com/s/bgz8p4dz5hcwn9d/java_bnd_chirps_subset.zip?dl=0) via gdalwarp
 - ```for i in `find *.tif`; do gdalwarp --config GDALWARP_IGNORE_BAD_CUTLINE YES -srcnodata NoData -dstnodata -9999 -cutline java_bnd_chirps_subset.shp -crop_to_cutline $i java_$i; done```
 
Clipped GeoTIFF file for Java (https://www.dropbox.com/sh/kgt1ixcbmuh0ovs/AAAnsDcl4ELOexG4Ft1mVlHMa?dl=0)

In [1]:
#!/usr/bin/env python
import numpy as np
import datetime as dt
import os
import gdal
import netCDF4
import re

In [2]:
ds = gdal.Open('/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.01.tif')

In [3]:
a = ds.ReadAsArray()
nlat,nlon = np.shape(a)

In [4]:
b = ds.GetGeoTransform() #bbox, interval
lon = np.arange(nlon)*b[1]+b[0]
lat = np.arange(nlat)*b[5]+b[3]

basedate = dt.datetime(1980,1,1,0,0,0)

In [5]:
# create NetCDF file
nco = netCDF4.Dataset('java_cli_chirps_1months_1981_2020.nc','w',clobber=True)

# create dimensions, variables and attributes:
nco.createDimension('lon',nlon)
nco.createDimension('lat',nlat)
nco.createDimension('time',None)

timeo = nco.createVariable('time','f4',('time'))
timeo.units = 'days since 1980-1-1 00:00:00'
timeo.standard_name = 'time'
timeo.calendar = 'gregorian'
timeo.axis = 'T'

lono = nco.createVariable('lon','f4',('lon'))
lono.units = 'degrees_east'
lono.standard_name = 'longitude'
lono.long_name = 'longitude'
lono.axis = 'X'

lato = nco.createVariable('lat','f4',('lat'))
lato.units = 'degrees_north'
lato.standard_name = 'latitude'
lato.long_name = 'latitude'
lato.axis = 'Y'

# create container variable for CRS: lon/lat WGS84 datum
crso = nco.createVariable('crs','i4')
crso.long_name = 'Lon/Lat Coords in WGS84'
crso.grid_mapping_name='latitude_longitude'
crso.longitude_of_prime_meridian = 0.0
crso.semi_major_axis = 6378137.0
crso.inverse_flattening = 298.257223563

# create short integer variable for precipitation data, with chunking
pcpo = nco.createVariable('precip', 'f4',  ('time', 'lat', 'lon'),zlib=True,fill_value=-9999.)
pcpo.units = 'mm'
pcpo.standard_name = 'convective precipitation rate'
pcpo.long_name = 'Climate Hazards group InfraRed Precipitation with Stations'
pcpo.time_step = 'month'
pcpo.missing_value = -9999.
pcpo.geospatial_lat_min = -8.8
pcpo.geospatial_lat_max = -5.05
pcpo.geospatial_lon_min = 105.05
pcpo.geospatial_lon_max = 116.25
pcpo.grid_mapping = 'crs'
pcpo.set_auto_maskandscale(False)

nco.Conventions='CF-1.6'
nco.title = "CHIRPS v2.0"
nco.history = "created by Climate Hazards Group. University of California at Santa Barbara"
nco.version = "Version 2.0"
nco.comments = "time variable denotes the first day of the given dekad."
nco.website = "https://www.chc.ucsb.edu/data/chirps"
nco.date_created = "2021-01-08"
nco.creator_name = "Benny Istanto"
nco.creator_email = "benny.istanto@wfp.org"
nco.institution = "UN World Food Programme"
nco.note = "The data is developed to support regular updating procedure for SPI analysis (https://github.com/wfpidn/SPI). This activities will support WFP to assess extreme dry and wet periods as part of WFP's Seasonal Monitoring"


#write lon,lat
lono[:]=lon
lato[:]=lat

pat = re.compile('java_chirps-v2.0.[0-9]{4}\.[0-9]{2}')
itime=0

#step through data, writing time and data to NetCDF
for root, dirs, files in os.walk('/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/'):
    dirs.sort()
    files.sort()
    for f in files:
        if re.match(pat,f):
            # read the time values by parsing the filename
            year=int(f[17:21])
            mon=int(f[22:24])
            date=dt.datetime(year,mon,1,0,0,0)
            print(date)
            dtime=(date-basedate).total_seconds()/86400.
            timeo[itime]=dtime
           # precipitation
            pcp_path = os.path.join(root,f)
            print(pcp_path)
            pcp=gdal.Open(pcp_path)
            a=pcp.ReadAsArray()  #data
            pcpo[itime,:,:]=a
            itime=itime+1

nco.close()

1981-01-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.01.tif
1981-02-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.02.tif
1981-03-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.03.tif
1981-04-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.04.tif
1981-05-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.05.tif
1981-06-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.06.tif
1981-07-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.07.tif
1981-08-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.08.tif
1981-09-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.09.tif
1981-10-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1981.10.tif
1981-11-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/

/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1993.08.tif
1993-09-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1993.09.tif
1993-10-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1993.10.tif
1993-11-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1993.11.tif
1993-12-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1993.12.tif
1994-01-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1994.01.tif
1994-02-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1994.02.tif
1994-03-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1994.03.tif
1994-04-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1994.04.tif
1994-05-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.1994.05.tif
1994-06-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.

2005-01-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.01.tif
2005-02-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.02.tif
2005-03-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.03.tif
2005-04-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.04.tif
2005-05-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.05.tif
2005-06-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.06.tif
2005-07-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.07.tif
2005-08-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.08.tif
2005-09-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.09.tif
2005-10-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2005.10.tif
2005-11-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/

2012-07-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2012.07.tif
2012-08-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2012.08.tif
2012-09-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2012.09.tif
2012-10-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2012.10.tif
2012-11-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2012.11.tif
2012-12-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2012.12.tif
2013-01-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2013.01.tif
2013-02-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2013.02.tif
2013-03-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2013.03.tif
2013-04-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2013.04.tif
2013-05-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/

2020-02-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.02.tif
2020-03-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.03.tif
2020-04-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.04.tif
2020-05-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.05.tif
2020-06-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.06.tif
2020-07-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.07.tif
2020-08-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.08.tif
2020-09-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.09.tif
2020-10-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.10.tif
2020-11-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/Java/java_chirps-v2.0.2020.11.tif
2020-12-01 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/